<a href="https://colab.research.google.com/github/dougscohen/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
""" IMPORTS """

import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Nadam

In [0]:
# Load the data

df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
df.dtypes

In [0]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')

In [0]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,2
top,9365-CSLBQ,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,5174


In [0]:
df = df.drop(columns='customerID')

df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
cols_to_encode = df.describe(exclude='number').columns.tolist()

In [0]:
encoder = OrdinalEncoder()

encoded = encoder.fit_transform(df[cols_to_encode])

In [0]:
df_test = pd.DataFrame(data=encoded, columns=cols_to_encode)

In [0]:
df_test.head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0


In [0]:
df = df.drop(columns=cols_to_encode)

df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65


In [0]:
df = pd.concat([df, df_test], axis=1)

In [0]:
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,0,1,29.85,29.85,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
1,0,34,56.95,1889.50,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0
2,0,2,53.85,108.15,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
3,0,45,42.30,1840.75,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,2,70.70,151.65,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0


In [0]:
df['Churn'] = df['Churn'].astype(int)

In [0]:
df.isnull().sum()

SeniorCitizen        0
tenure               0
MonthlyCharges       0
TotalCharges        11
gender               0
Partner              0
Dependents           0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
Churn                0
dtype: int64

In [0]:
df = df.fillna(df['TotalCharges'].mean())

In [0]:
df.isnull().sum()

SeniorCitizen       0
tenure              0
MonthlyCharges      0
TotalCharges        0
gender              0
Partner             0
Dependents          0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Churn               0
dtype: int64

In [0]:
train, test = train_test_split(df, test_size=0.2, random_state=3)

train.shape, test.shape

((5634, 20), (1409, 20))

In [0]:
target = 'Churn'
features = df.columns[:-1].tolist()

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [0]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [0]:
# Create and fit a baseline Keras MLP model to the data

model = Sequential([
    Dense(10, input_shape=(19,), activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(
    x = X_train_scaled,
    y = y_train,
    validation_data=(X_test_scaled, y_test),
    epochs = 10
)

Epoch 1/10
177/177 [==============================] - 0s 3ms/step - loss: 0.6557 - accuracy: 0.6335 - val_loss: 0.5423 - val_accuracy: 0.7367
Epoch 2/10
177/177 [==============================] - 0s 2ms/step - loss: 0.5301 - accuracy: 0.7327 - val_loss: 0.4799 - val_accuracy: 0.7722
Epoch 3/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4891 - accuracy: 0.7609 - val_loss: 0.4503 - val_accuracy: 0.7878
Epoch 4/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4675 - accuracy: 0.7749 - val_loss: 0.4341 - val_accuracy: 0.7977
Epoch 5/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4550 - accuracy: 0.7833 - val_loss: 0.4243 - val_accuracy: 0.7977
Epoch 6/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4463 - accuracy: 0.7874 - val_loss: 0.4176 - val_accuracy: 0.7999
Epoch 7/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4397 - accuracy: 0.7938 - val_loss: 0.4126 - val_accuracy: 0.8034
Epoch 

In [0]:
# Hyperperameter tuning using Grid Search

###############
# BATCH SIZES #
###############

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7958802342414856 using {'batch_size': 10, 'epochs': 20}
Means: 0.7958802342414856, Stdev: 0.007393975611929873 with: {'batch_size': 10, 'epochs': 20}
Means: 0.793929398059845, Stdev: 0.00321357012775786 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7898449301719666, Stdev: 0.008599543093119312 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7880674839019776, Stdev: 0.014073387545211676 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7919744849205017, Stdev: 0.010431998922828308 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7900211334228515, Stdev: 0.01166435817137094 with: {'batch_size': 100, 'epochs': 20}


In [0]:
# Hyperperameter tuning using Grid Search

##############
# OPTIMIZERS #
##############

# Function to create model, required for KerasClassifier
def create_model(opt='adam'):
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10],
              'epochs': [20],
              'opt': ['sgd', 'adam', 'nadam']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7935724258422852 using {'batch_size': 10, 'epochs': 20, 'opt': 'nadam'}
Means: 0.7921495795249939, Stdev: 0.01527825031595987 with: {'batch_size': 10, 'epochs': 20, 'opt': 'sgd'}
Means: 0.7917979598045349, Stdev: 0.005780747443582631 with: {'batch_size': 10, 'epochs': 20, 'opt': 'adam'}
Means: 0.7935724258422852, Stdev: 0.007751237474783576 with: {'batch_size': 10, 'epochs': 20, 'opt': 'nadam'}


In [0]:
# Hyperperameter tuning using Grid Search

##################
# LEARNING RATES #
##################

# Function to create model, required for KerasClassifier
def create_model(lr=.01):
    # create model
    opt = Nadam(learning_rate=lr)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10],
              'epochs': [20],
              'lr': [.001, .01, .1, .2]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.7951700806617736 using {'batch_size': 10, 'epochs': 20, 'lr': 0.001}
Means: 0.7951700806617736, Stdev: 0.007126818755074147 with: {'batch_size': 10, 'epochs': 20, 'lr': 0.001}
Means: 0.7893144369125367, Stdev: 0.007657085834726442 with: {'batch_size': 10, 'epochs': 20, 'lr': 0.01}
Means: 0.7935730576515198, Stdev: 0.007773234440030202 with: {'batch_size': 10, 'epochs': 20, 'lr': 0.1}
Means: 0.7325138926506043, Stdev: 0.011067145609281617 with: {'batch_size': 10, 'epochs': 20, 'lr': 0.2}


In [0]:
# Hyperperameter tuning using Grid Search

########################
# ACTIVATION FUNCTIONS #
########################

# Function to create model, required for KerasClassifier
def create_model(act='relu'):
    # create model
    opt = Nadam(learning_rate=.001)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation=act))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [20],
              'act': ['sigmoid', 'elu', 'relu', 'selu', 'softplus', 'tanh']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7974764585494996 using {'act': 'sigmoid', 'batch_size': 20, 'epochs': 20}
Means: 0.7974764585494996, Stdev: 0.009553011062620005 with: {'act': 'sigmoid', 'batch_size': 20, 'epochs': 20}
Means: 0.7932170510292054, Stdev: 0.0087082174584839 with: {'act': 'elu', 'batch_size': 20, 'epochs': 20}
Means: 0.7917962312698364, Stdev: 0.011191853785749033 with: {'act': 'relu', 'batch_size': 20, 'epochs': 20}
Means: 0.7960564255714416, Stdev: 0.00983122222022535 with: {'act': 'selu', 'batch_size': 20, 'epochs': 20}
Means: 0.7964123010635376, Stdev: 0.007181909819161906 with: {'act': 'softplus', 'batch_size': 20, 'epochs': 20}
Means: 0.795169448852539, Stdev: 0.008496110947419285 with: {'act': 'tanh', 'batch_size': 20, 'epochs': 20}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?